In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# read and and preprocess the groundtruth

with open('./groundtruth_train.txt', 'r') as file:
    
    data = file.read()
    data = data.split('\n')
    data = [np.array(list(map(float, i.split()[1:4]))) for i in data]
    data = data[:-1]
    data = np.vstack(data)

In [ ]:
def TUM_EVAL(points, targets):
    '''
        Estimates least squares parameters for 3D transformation between two sets of points
        
        Ref:https://graphics.stanford.edu/courses/cs164-09-spring/Handouts/paper_Umeyama.pdf
        Equations 40, 41, 42 
    '''
    
    
    
    # print(points.shape, targets.shape)
    assert points.shape == targets.shape
    
    pmean, tmean = np.mean(points, axis=0), np.mean(targets, axis=0)
    # tvar = np.mean(np.linalg.norm(targets - tmean, axis=1) ** 2)
    
    H = (points-pmean).T@(targets-tmean)
    U, S, V = np.linalg.svd(H)
    V= V.T
    S = np.diag(S)
    
    R = V@U.T
    if np.linalg.det(R)<0:
        V[:, 2] = -1*V[:, 2]
        R = V@U.T
    
    mR_cA = -R@pmean.T
    
    A = points @R.T + mR_cA
    B = targets-tmean
    
    saa = 0
    sab = 0
    for i in range(A.shape[0]):
        saa += A[i, :]@A[i, :].T
        sab += A[i, :]@B[i, :].T
   
    scale = sab/saa
    t = scale*mR_cA + tmean.T
    # rmse = np.linalg.norm(scale*A-B)/np.sqrt(A.shape[0])
    
    return  scale, R, t

In [ ]:
path = np.load('./predictions.npy') # Predictions 

The predictions are separated into two segments, start and end segment. A tranformation is learned from both the segments to the ground truth and an alignment metric is caculated as follows

Let $p_i$ be the trackes positions and $\hat{p_i}$ be the ground truth

$$\begin{align}
    T_s^{gt} &= \min_{T \in Sim(3)} \sum_{i \in S} (T p_i - \hat p_i)^2\\
    T_e^{gt} &= \min_{T \in Sim(3)} \sum_{i \in E} (T p_i - \hat p_i)^2\\
    \\
    error_{align} &= \sqrt{\frac{1}{n} \sum_i^{n} \|T_s^{gt}p_i-T_e^{gt}p_i\|_2^2}
\end{align}$$

More about $Sim(3)$ matrices [here](https://www.ethaneade.org/latex2html/lie/node28.html)



In [ ]:
k = data.shape[0]//2
ind = np.where(~np.isnan(data).any(axis=1))[0]
segS = ind[ind<=k]
segL = ind[ind>k]

cs, Rs, ts = TUM_EVAL(path[segS], data[segS])
ce, Re, te = TUM_EVAL(path[segL], data[segL])

In [ ]:
predS = cs*path@Rs.T + ts.T
predE = ce*path@Re.T + te.T

In [ ]:
error = np.linalg.norm(predS-predE)/np.sqrt(data.shape[0])
error # Error used to rank the submissions